**Procedures and Themes**

In [ ]:
#some tasks of banks and their descriptions :
#Query Type Requesting Static Doc:
# displaying current available loans to the user (which depend on the eligibility criteria of the user, credit scores and stuff)
# Displaying the status of loan accounts (which is created after loan is approved and terminates when the debt is cleared all the while contianing info about how much is paid, how muich is due and all the stuff)
# Display all the details linked with his/her account like pan card, address proofs, different accounts holding and the money being held, credit or cibil scores, current debts, current income on FDs, past transactions (account statements) etc
# Displaying bank policies, key term definitions

#Query Type Requesting Dynamic Process(Either interactive chat or video suggestions with some involving executives for security purposes):
# Creating an account (based on demographics of the user recommend account types)
# Changing account or contact details
# Applying for loan
# Foreclosing Loan procedures
# Applying for Credit/Debit cards
# Applying for Passbooks, Cheque Books, Internet Banking
# Applying for PIN change
# Closure of Bank accounts with phone call verification
# Others 

**Libraries Import**

In [ ]:
!pip install gradio
!pip install SpeechRecognition
!pip install gTTS
!pip install transformers 
!pip install tensorflow
!pip install tensorflow-text
!pip install flair==0.4.3 
!pip install faiss-cpu
#!curl -L https://anaconda.org/pytorch/faiss-cpu/1.6.0/download/linux-64/faiss-cpu-1.6.0-py36h6bb024c_0.tar.bz2 | tar xj
#!mv lib/python3.6/site-packages/* /usr/local/lib/python3.6/dist-packages/


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.1 MB 6.9 MB/s 
     |████████████████████████████████| 212 kB 53.7 MB/s 
     |████████████████████████████████| 57 kB 6.2 MB/s 
     |████████████████████████████████| 54 kB 4.1 MB/s 
     |████████████████████████████████| 1.1 MB 58.7 MB/s 
     |████████████████████████████████| 2.3 MB 39.4 MB/s 
     |████████████████████████████████| 256 kB 69.0 MB/s 
     |████████████████████████████████| 140 kB 50.5 MB/s 
     |████████████████████████████████| 84 kB 4.8 MB/s 
     |████████████████████████████████| 94 kB 4.3 MB/s 
     |████████████████████████████████| 144 kB 59.2 MB/s 
     |████████████████████████████████| 271 kB 62.6 MB/s 
     |████████████████████████████████| 63 kB 2.7 MB/s 
     |████████████████████████████████| 80 kB 10.5 MB/s 
     |████████████████████████████████| 43 kB 2.6 MB/s 
     |████████████████████████████████| 62 kB

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.6 MB 6.3 MB/s 


In [ ]:
!pip install django
!django-admin startproject resonancebrains

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.9 MB 4.1 MB/s 


In [ ]:
%cd resonancebrains/
!ls


/content/resonancebrains
manage.py  resonancebrains


In [ ]:
from abc import ABCMeta, abstractmethod

import pandas as pd
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow_text

import faiss, os

from tqdm import tqdm
import numpy as np
import transformers
from transformers import pipeline
import ast, json, re
from sklearn.metrics.pairwise import cosine_similarity
import requests

**Video Semantic Search and Static Query Search Code Base**

In [ ]:
class TFEncoder(metaclass=ABCMeta):
    """Base encoder to be used for all encoders."""
    def __init__(self, model_path:str):
        self.model = hub.load(model_path)
    
    @abstractmethod
    def encode(self, text:list):
        """Encodes text.
        Text: should be a list of strings to encode
        """

class USE(TFEncoder):
    """Universal sentence encoder"""
    def __init__(self, model_path):
        super().__init__(model_path)
        
    def encode(self, text):
        return self.model(text).numpy()


model_path = "https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3"



class FAISS:
    def __init__(self, dimensions:int):
        self.dimensions = dimensions
        self.index = faiss.IndexFlatL2(dimensions)
        self.snips = {}
        self.snip_id = 0

    
    def add(self, text:str, v:list, text_details:tuple):
        self.index.add(v)
        snip_details = (text,)+text_details
        print(snip_details)
        self.snips[self.snip_id] = snip_details
        self.snip_id += 1
        
    def search(self, v:list, k:int=10):
        distance, item_index = self.index.search(v, k)
        results = []
        for dist, i in zip(distance[0], item_index[0]):
            if i==-1:
                break
            else:
              results.append(self.snips[i][:-1]+(dist,)) 
                
        return results

    def __get_snip_id(self, v):
      try:
        ind = [k for k, vl in self.vectors.items() if v==vl[0]][0]
        return ind
      except:
        return -1

class VideoSemanticSearch(FAISS):
  def __init__(self, semantic_outlier_radius):
    self.encoder = USE(model_path)
    self.d = self.encoder.encode(['hello']).shape[-1]
    super().__init__(self.d)
    self.semantic_outlier_radius = semantic_outlier_radius

  def add(self, url, handle, v):
    self.index.add(v)
    snip_details = (url, handle, v)
    self.snips[self.snip_id] = snip_details
    self.snip_id += 1
  
  def add_pool(self, filepaths=[]):
    
    def isEnglish(s):
      try:
          s.encode(encoding='utf-8').decode('ascii')
      except UnicodeDecodeError:
          return False
      else:
          return True
      
    for path in filepaths:
      df = pd.read_csv(path)
      handles = df['Video Name'].values
      for handle, url in df[['Video Name', 'Video URL']].values:
        if type(handle) != float and type(url) != float:
          refined_handle = re.sub("bajaj|finserv", "", re.sub("([0-9\W]*)(?!\w)", "",  re.sub("\|.*", "", handle.lower()))).strip()
          if isEnglish(refined_handle):
            emb = self.encoder.encode(refined_handle)
            self.add(url, refined_handle, emb)
    return 1


  def search_snip(self, s, k=10):
      emb = self.encoder.encode(s)
      res = self.search(emb, k)
      res = [r for r in res if r[-1]<=self.semantic_outlier_radius]
      if len(res) == 0:
        return [("Sorry no appropriate response could solve your query try framing a different query", 1e+7)]
      return res


class StaticQuerySearch(FAISS):
  def __init__(self, semantic_outlier_radius):
    self.encoder = USE(model_path)
    self.d = self.encoder.encode(['hello']).shape[-1]
    super().__init__(self.d)
    self.semantic_outlier_radius = semantic_outlier_radius
    
  def add(self, v, response):
    self.index.add(v)
    snip_details = (response,v)
    self.snips[self.snip_id] = snip_details
    self.snip_id += 1
  
  def add_pool(self, filepaths=[]):
    
    dfs = []
    for path in filepaths:
      try:
        dfs.append(pd.read_csv(path))
      except:
        dfs.append(pd.read_csv(path, encoding="ISO-8859-1"))
    
    for df in tqdm(dfs):
      for rowd in df.values:
        if type(rowd[0]) != float and type(rowd[1]) != float:
          text, response = rowd[0].replace("?", ""), rowd[1]#, rowd[3] 
          emb = self.encoder.encode(text)
          self.add(emb, response)
    return 1
  
  def search_snip(self, s, k=10):

      emb = self.encoder.encode(s.replace("?", ""))
      res = self.search(emb, k)
      res = [r for r in res if r[-1]<=self.semantic_outlier_radius]  #recommendations in radius of 100 euclideans
      if len(res) == 0:
        return [("Sorry no appropriate response could solve your query try framing a different query", 1e+7)]
      return res

class SmallTalkSearch(FAISS):
  def __init__(self, semantic_outlier_radius):
    self.encoder = USE(model_path)
    self.d = self.encoder.encode(['hello']).shape[-1]
    super().__init__(self.d)
    self.semantic_outlier_radius = semantic_outlier_radius
    
    
  def add(self, v, response,):
    self.index.add(v)
    snip_details = (response, v)
    self.snips[self.snip_id] = snip_details
    self.snip_id += 1
  
  def add_pool(self, ques=[]):
    
    for q in tqdm(ques):
        
        text = q.lower() 
        emb = self.encoder.encode(text)
        self.add(emb, text)
    return 1
  
  def search_snip(self, s, k=10):

      emb = self.encoder.encode(s)
      res = self.search(emb, k)
      res = [r for r in res if r[-1]<=self.semantic_outlier_radius]  
      if len(res) == 0:
        return [("Sorry no appropriate response could solve your query try framing a different query", 1e+5)]
      return res





class ChatBot():
  def __init__(self):
    self.vsearch_engine = VideoSemanticSearch(1) #radius=1
    self.rsearch_engine = StaticQuerySearch(1) #radius=1
    self.stsearch_engine = SmallTalkSearch(1)
    self.vsearch_engine.add_pool(filepaths=["./Video_Final_DB - Sheet1.csv","./allianz_youtube - Sheet1.csv"]) #filepaths=['/content/keywords_timestamps.txt', '/content/transcript_timestamps.txt'])
    self.rsearch_engine.add_pool(filepaths=['./general_health_insurance_faqs_Sheet1.csv', "./car insurance faqs.csv", "./travel insurance faqs.csv"])
    self.small_talk = ['Hi',
 'Hello',
 'How are you?',
 'What’s up?',
 'Yo',
 'How you doin?',
 'Good Morning',
 'Good Evening',
 'Good Afternoon',
 'Good Night',
 'What can you do?',
 'What can I do with you?',
 'What are you doing?',
 'Thanks',
 'Thank you',
 'Thanks dude',
 'Thanks lady',
 'My name is &lt;insert user name&gt;',
 'How can you help me?',
 'Help me please',
 'Talk to you later!',
 'Tell me something that you can do',
 'Say anything',
 'Will you marry me?',
 'Are you Single?',
 'Do you have a boyfriend',
 'girlfriend?',
 'Do you know other chatbots?',
 'Do you want to rule the world?',
 'Are you busy?',
 'Say something funny',
 'Can you say anything else',
 'What do you think of me?',
 'Happy &lt;insert holiday&gt;',
 'Do you love me?',
 'Can you fly?',
 'You are a genius!',
 'You are awesome',
 'great',
 'special',
 'Thanks so much!',
 'Thanks, that’s very helpful',
 'Yup, that’s true',
 'That’s accurate',
 'You are right',
 'Give me a fist bump',
 'I think you are great!',
 'Thumbs up',
 'You suck',
 'I hate you',
 'Go away',
 'That’s not right',
 'That’s not true',
 'That was stupid',
 'You are not helpful',
 'That was not the right answer',
 'Terrible',
 '&lt;expletive&gt;',
 'Which baseball',
 'football team do you like?',
 'You know nothing Jon Snow.',
 'Where’s Waldo?',
 'What’s the weather?',
 'Where’s the best restaurant in town?',
 'What should I eat today?',
 'Who is the best &lt;insert role&gt;?',
 'Who should I vote for?',
 'Who is the best digital assistant?',
 'Do you know Alexa?',
 'What’s your age?',
 'Can you sleep?',
 'Who is your boss?',
 'Who created you?',
 'Do you know other chatbots?',
 'What is your favorite color?',
 'Why did the chicken cross the road?',
 'Happy Birthday',
 'Do you know me?',
 'Are you my assistant?']
    self.stsearch_engine.add_pool(ques=self.small_talk)
    self.conversor = pipeline("conversational", model="microsoft/DialoGPT-medium")
    os.environ["TOKENIZERS_PARALLELISM"] = "true"
    

  
  import requests
  def search(self, name):
      url = "https://google-web-search.p.rapidapi.com/"

      querystring = {"query":name,"max":"1","site":"https://www.bajajallianz.com/"}

      headers = {
          "X-RapidAPI-Key": "00d6ea68d4msha77ac50f2364dd8p10dabfjsn21c29d49dc02",
          "X-RapidAPI-Host": "google-web-search.p.rapidapi.com"
      }

      response = requests.request("GET", url, headers=headers, params=querystring)
      res = response.json()
      final_des = res["results"][0]["description"].replace("\xa0..", "")
      url = res["results"][0]["url"]
      return final_des + f"refer this url for more details : {url}"

  
  def chat(self, usr_chat):

    def decider(source_distance, choices_distance, threshold_distance=5e-2):
      proximity_array = []
      for d in range(choices_distance.shape[0]):
        
        if abs(source_distance - choices_distance[d]) <= threshold_distance:
          proximity_array.append(d)
        else:
          pass
      return proximity_array
        
    try:
    
      usr_chat = usr_chat.lower()
      

      formal_text_resp = self.rsearch_engine.search_snip(usr_chat, 5)
      informal_text_resp = self.stsearch_engine.search_snip(usr_chat,1)[0]
      video_resp = self.vsearch_engine.search_snip(usr_chat, 5)

      formal_text_resp, recommend_formal_text_resp_next = formal_text_resp[0], formal_text_resp[1:]
      video_respf, recommend_video_resp_next = video_resp[0], video_resp[1:]
      
      informal_text_resp_text = self.conversor(transformers.Conversation(usr_chat), pad_token_id=50256)
      informal_text_resp_text = str(informal_text_resp_text)
      informal_text_resp_text = informal_text_resp_text[informal_text_resp_text.find("bot >> ")+6:].strip()

      
      total_dist = np.array([formal_text_resp[-1], informal_text_resp[-1], video_respf[-1]])      
      total_resp_text = np.array([formal_text_resp[0], informal_text_resp_text, video_respf[0]])
      

      source_distance = np.min(total_dist)
      decided_categories = decider(source_distance, total_dist)

      
      #print(total_resp_text, decided_categories)
      if 1 in decided_categories:
        decided_categories = [1]
      else:
        pass
        

      if (0 in decided_categories):
        final_resp = {"yt_id":"", "flag":True, "text":total_resp_text[0]}
      elif (1 in decided_categories):
        final_resp = {"yt_id":"", "flag":True, "text":total_resp_text[0]}
        final_resp["text"] = self.search(usr_chat)
      else:
        ytid = re.findall("/watch\?v=(.*)&list", total_resp_text[2])[0]
        final_resp = {"yt_id":ytid, "flag":False, "text":""}
      
      return {"status": 1, "response":json.dumps(final_resp)}
    except Exception as e:
      print(e)
      return {"status" : 0, "response":"Sorry we couldn't understand the query try framing it differently"}
      
                        

In [ ]:
bot = ChatBot() 
bot.chat("hi")

100%|██████████| 79/79 [00:04<00:00, 17.41it/s]


Downloading:   0%|          | 0.00/642 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/823M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

{'response': '{"yt_id": "", "flag": true, "text": "You can renew your insurance plan by visiting our website and by using our mobile app \\u2013 Caringly Yours. You can also send us a \'Hi\' on our WhatsApp number (+91.refer this url for more details : https://www.bajajallianz.com/blog/announcements/digital-care-for-you-in-the-digital-age.html"}',
 'status': 1}

In [ ]:
bot.chat("what are mots")

{'response': '{"yt_id": "", "flag": true, "text": "Claim settlement within 20 minutes* after completion of all process. Claim settlement depends upon the number of days taken for vehicle repairs and bill.refer this url for more details : https://www.bajajallianz.com/motor-insurance/ots-car-insurance-claim-process.html"}',
 'status': 1}

In [ ]:
bot.chat("what are different steps for insurance")
#bot.recommend_chat_queries(1)

{'response': '{"yt_id": "", "flag": true, "text": "Follow the procedure to get car insurance online:\\n\\n#1 For new car insurance click on \\u0093Get a Quote.\\u0094\\n\\n#2 To renew the existing policy, click on Renew.\\n\\n#3 Fill in the vehicle and your details.\\n\\n#4 Select a quote.\\n\\n#5 Pay the said amount, and the policy will be emailed in the pdf format. "}',
 'status': 1}

In [ ]:
bot.chat("are you crazy")  #

{'status': 1,
 'response': '{"yt_id": "", "flag": true, "text": "I\'m not crazy, I\'m just a little crazy."}'}

In [ ]:
import pandas as pd
pd.read_csv("/content/car insurance faqs.csv", encoding="ISO-8859-1")

,Questions,Answers
0,NaN,NaN
1,Why do I need to insure my vehicle?,The Motor Vehicle Act of 1988 mandates that ev...
2,How much fine do I have to pay in case I do no...,The first time offence of driving without car ...
3,What are the types of car insurance in India?,"Out of the two types of car insurance, Third-P..."
4,What are the things to remember before buying ...,Select a reputed insurance provider followed b...
...,...,...
90,What to do in case of an Own Damage Claim?,First and foremost inform the police and docum...
91,What is Constructive Total Loss in car insuran...,The term CTL is accorded to a situation when t...
92,How many times can we claim four wheeler insur...,It depends upon the car insurance company or o...
93,How can I cancel my claim?,You can contact your car insurance company or ...


In [ ]:
bot.chat("when")

{'status': 1,
 'response': '{"yt_id": "", "flag": true, "text": "Claim settlement within 20 minutes* after completion of all process. Claim settlement depends upon the number of days taken for vehicle repairs and bill.refer this url for more details : https://www.bajajallianz.com/motor-insurance/ots-car-insurance-claim-process.html"}'}

In [ ]:
json.loads(bot.chat("what is MOTS")["response"])["text"]

'Claim settlement within 20 minutes* after completion of all process. Claim settlement depends upon the number of days taken for vehicle repairs and bill.refer this url for more details : https://www.bajajallianz.com/motor-insurance/ots-car-insurance-claim-process.html'

In [ ]:
json.loads(bot.chat("ways to save taxes")["response"])["text"] 

'Bajaj Allianz General Insurance Health Insurance policy helps you save tax up to INR 1 lakh under Section 80D against the premiums you pay. Here’s how you can save tax:On the premiums you pay for yourself, your spouse, children and parents, you can avail INR 25,000 per annum as a deduction against your taxes (provided you are not over 60 years). If you pay a premium for your parents who are senior citizens (age 60 or above), the maximum health insurance benefit for tax purposes is capped at INR 50,000. As a taxpayer, you may, therefore, maximize tax benefit under Section 80D up to a total of INR 75,000, if you are below 60 years of age and your parents are senior citizens. If you are above the age of 60 years and are paying a medical insurance premium for your parents, the maximum tax benefit under Section 80D is, then, INR 1 lakh.'

In [ ]:
json.loads(bot.chat("growth story of bajaj allianz general insurance")["response"]) 

{'yt_id': 'XygLDK_t7ok', 'flag': False, 'text': ''}

In [ ]:
json.loads(bot.chat("should i have both health and life insurance")["response"])["text"]  #bot.chat("should i have both health and life insurance")

'Life insurance is a great investment and death cover option but it won’t come in handy to pay rising medical bills. A Health Insurance cover is a solution to deal with hospitalization and pay heavy medical expenses, should they arise. So, Health Insurance plans are your cover against unexpected health-related costs that may threaten to deplete your savings.'

In [ ]:
json.loads(bot.chat("can i buy more than one health policy")["response"])["text"]

'Yes, you can buy more than one health insurance policies. However, it usually becomes cumbersome to handle multiple policies. We recommend you to buy a single health insurance policy with high SI instead of buying multiple health insurance policies with low SI.'

In [ ]:
cost_mapping  = {"slight_scratch":1000,"severe_scratch":100,"slight_deformation":10000,"medium_deformation":10000,"severe_deformation":10000,"crack_and_hole":8000,"fender/headlight_damage":5000,"windshield_damage":8000 ,"car_light_crack":8000,"car_light_severe_crack":8000,"car_light_damage":8000,"fender/bumper_damage":5000,"side_mirror_scratch":2000,"side_mirror_crack":2000,"side_mirror_drop_off":2000}

In [ ]:
st = 'Claim settlement within 20 minutes* after completion of all process. claims are best. Claim settlement depends upon the number of days taken for vehicle repairs and bill\xa0...'

search("I want to vehicle insurance for my car")["results"][0]["description"]

'Car Insurance Online by Bajaj Allianz. Buy Car Insurance Policy With 24x7 Assistance, 15 Lakh PA Cover, 6000+ Cashless Garages, Quick Claims\xa0...'

In [ ]:
import requests
def search(name):
    url = "https://google-web-search.p.rapidapi.com/"

    querystring = {"query":name,"max":"1","site":"https://www.bajajallianz.com/"}

    headers = {
        "X-RapidAPI-Key": "00d6ea68d4msha77ac50f2364dd8p10dabfjsn21c29d49dc02",
        "X-RapidAPI-Host": "google-web-search.p.rapidapi.com"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)
    res = response.json()
    final_des = res["results"][0]["description"].replace("\xa0", "")
    url = res["results"][0]["url"]
    return final_des + f"\n refer this url for more details : {url}"

print(search("I want to vehicle insurance for my car"))

Car Insurance Online by Bajaj Allianz. Buy Car Insurance Policy With 24x7 Assistance, 15 Lakh PA Cover, 6000+ Cashless Garages, Quick Claims...
 refer this url for more details : https://www.bajajallianz.com/motor-insurance/car-insurance-online.html


In [ ]:
damage_details = {
  "result": {
    "draw_result": True,
    "job_id": "33c3e6e8-85cd-467a-b2ea-28088db90f27",
    "output": {
      "elements": [
        {
          "bbox": [
            472,
            443,
            615,
            498
          ],
          "damage_category": "slight_scratch",
          "damage_color": [
            50,
            50,
            100
          ],
          "damage_id": "1",
          "damage_location": "left_front_door",
          "score": 0.301284
        },
        {
          "bbox": [
            459,
            603,
            616,
            669
          ],
          "damage_category": "severe_scratch",
          "damage_color": [
            0,
            0,
            120
          ],
          "damage_id": "2",
          "damage_location": "left_doorsill",
          "score": 0.376836
        },
        {
          "bbox": [
            434,
            592,
            615,
            682
          ],
          "damage_category": "medium_deformation",
          "damage_color": [
            0,
            115,
            255
          ],
          "damage_id": "4",
          "damage_location": "left_doorsill",
          "score": 0.437898
        },
        {
          "bbox": [
            369,
            166,
            777,
            634
          ],
          "damage_category": "severe_deformation",
          "damage_color": [
            0,
            95,
            255
          ],
          "damage_id": "5",
          "damage_location": "left_front_door",
          "score": 0.915449
        },
        {
          "bbox": [
            162,
            243,
            401,
            650
          ],
          "damage_category": "severe_deformation",
          "damage_color": [
            0,
            95,
            255
          ],
          "damage_id": "5",
          "damage_location": "left_front_wing",
          "score": 0.391524
        }
      ]
    },
    "output_url": "https://vpai-demo-portal-shanghai.oss-accelerate.aliyuncs.com/vehicle-damage%2Fdraw-103414.2508_20220625103414.jpg?OSSAccessKeyId=LTAI5tGgUWzx5gZopTcAvNtp&Expires=1656153314&Signature=6YwSLqWhn6LtgJsCKMi4eKn9M2Y%3D",
    "url_expiry": "2022/06/25 10:35:14"
  }
}

In [ ]:
arg = [{'draw_result': True, 'job_id': '79a7f62e-308d-4365-a984-1022676297a3', 'output': {'elements': [{'bbox': [892, 272, 993, 349], 'damage_category': 'slight_scratch', 'damage_color': [50, 50, 100], 'damage_id': '1', 'damage_location': 'rear_bumper', 'score': 0.62233}, {'bbox': [889, 
426, 960, 452], 'damage_category': 'slight_scratch', 'damage_color': [50, 50, 100], 'damage_id': '1', 'damage_location': 'rear_bumper', 'score': 0.391879}, {'bbox': [976, 213, 1003, 266], 'damage_category': 'slight_scratch', 'damage_color': [50, 50, 100], 'damage_id': '1', 'damage_location': 'rear_bumper', 'score': 0.306735}, {'bbox': [840, 476, 984, 538], 'damage_category': 'severe_scratch', 'damage_color': [0, 0, 120], 'damage_id': '2', 'damage_location': 'rear_bumper', 'score': 0.306434}, {'bbox': [582, 474, 955, 566], 'damage_category': 'medium_deformation', 'damage_color': [0, 115, 255], 'damage_id': '4', 'damage_location': 'rear_bumper', 'score': 0.870388}]}, 'output_url': 'https://vpai-demo-portal-shanghai.oss-accelerate.aliyuncs.com/vehicle-damage%2Fdraw-020203.3704_20220626020203.jpg?OSSAccessKeyId=LTAI5tGgUWzx5gZopTcAvNtp&Expires=1656208983&Signature=bcBbsAdrM2ctgeQ644huM8Plb9c%3D', 'url_expiry': '2022/06/26 02:03:03'}, {'draw_result': True, 'job_id': '34483165-78ef-46d6-bb0c-6f079e67171d', 'output': {'elements': [{'bbox': [871, 258, 980, 350], 'damage_category': 'slight_scratch', 'damage_color': [50, 50, 100], 'damage_id': '1', 'damage_location': 'rear_bumper', 'score': 0.5422}, {'bbox': [874, 413, 944, 444], 'damage_category': 'slight_scratch', 'damage_color': [50, 50, 100], 'damage_id': '1', 'damage_location': 'rear_bumper', 'score': 0.399309}, {'bbox': [814, 467, 971, 529], 'damage_category': 'severe_scratch', 'damage_color': [0, 0, 120], 'damage_id': '2', 'damage_location': 'rear_bumper', 'score': 0.502095}, {'bbox': [573, 464, 945, 560], 'damage_category': 'medium_deformation', 'damage_color': [0, 115, 255], 'damage_id': '4', 'damage_location': 'rear_bumper', 'score': 0.786156}]}, 'output_url': 'https://vpai-demo-portal-shanghai.oss-accelerate.aliyuncs.com/vehicle-damage%2Fdraw-020207.8693_20220626020207.jpg?OSSAccessKeyId=LTAI5tGgUWzx5gZopTcAvNtp&Expires=1656208988&Signature=EjzRjK2%2F9VbgbG52xm%2FcacL8B7w%3D', 'url_expiry': '2022/06/26 02:03:08'}, {'draw_result': True, 'job_id': 'a9fff642-4280-48cc-9448-22f56fcb0e5e', 'output': {'elements': [{'bbox': [725, 395, 818, 435], 'damage_category': 'slight_scratch', 'damage_color': [50, 50, 100], 'damage_id': '1', 'damage_location': 'rear_bumper', 'score': 
0.589529}, {'bbox': [697, 198, 829, 339], 'damage_category': 'slight_scratch', 'damage_color': [50, 50, 100], 'damage_id': '1', 'damage_location': 'right_tail_light', 'score': 0.33738}, {'bbox': [691, 340, 722, 364], 'damage_category': 'slight_scratch', 'damage_color': [50, 50, 100], 'damage_id': '1', 'damage_location': 'rear_bumper', 'score': 0.32482}, {'bbox': [720, 185, 832, 337], 'damage_category': 'severe_scratch', 'damage_color': [0, 0, 120], 'damage_id': '2', 'damage_location': 'right_tail_light', 'score': 0.406893}, {'bbox': [759, 460, 819, 501], 'damage_category': 'severe_scratch', 'damage_color': [0, 0, 120], 'damage_id': '2', 'damage_location': 'rear_bumper', 'score': 0.306026}, {'bbox': [449, 460, 816, 574], 'damage_category': 'medium_deformation', 'damage_color': [0, 115, 255], 'damage_id': '4', 'damage_location': 'rear_bumper', 'score': 0.845764}, {'bbox': [451, 485, 630, 580], 'damage_category': 'medium_deformation', 'damage_color': [0, 115, 255], 'damage_id': '4', 'damage_location': 'rear_bumper', 'score': 0.472864}]}, 'output_url': 'https://vpai-demo-portal-shanghai.oss-accelerate.aliyuncs.com/vehicle-damage%2Fdraw-020211.9252_20220626020211.jpg?OSSAccessKeyId=LTAI5tGgUWzx5gZopTcAvNtp&Expires=1656208992&Signature=RQaJAq6uOgPhIQHDZArP%2BscEh40%3D', 'url_expiry': '2022/06/26 02:03:12'}, {'draw_result': True, 'job_id': '2a1d80e0-65fe-46bc-a82a-674b666ee78d', 'output': {'elements': [{'bbox': [331, 443, 472, 480], 'damage_category': 'slight_scratch', 'damage_color': [50, 50, 100], 'damage_id': '1', 'damage_location': 'rear_bumper', 'score': 0.707205}, {'bbox': [348, 532, 411, 602], 'damage_category': 'slight_scratch', 'damage_color': [50, 50, 100], 'damage_id': '1', 'damage_location': 'rear_bumper', 'score': 0.557061}, {'bbox': [937, 153, 973, 206], 'damage_category': 'slight_scratch', 'damage_color': [50, 50, 100], 'damage_id': '1', 'damage_location': 'right_rear_wing', 'score': 0.488187}, {'bbox': [922, 151, 989, 230], 'damage_category': 'slight_scratch', 'damage_color': [50, 50, 100], 'damage_id': '1', 'damage_location': 'right_rear_wing', 'score': 0.429818}, {'bbox': [630, 474, 682, 505], 'damage_category': 'slight_scratch', 'damage_color': [50, 50, 100], 'damage_id': '1', 'damage_location': 'rear_bumper', 'score': 0.41435}, {'bbox': [936, 96, 999, 216], 'damage_category': 'slight_scratch', 'damage_color': [50, 50, 100], 'damage_id': '1', 
'damage_location': 'right_rear_wing', 'score': 0.340294}, {'bbox': [866, 226, 923, 296], 'damage_category': 'slight_scratch', 'damage_color': [50, 50, 100], 'damage_id': '1', 'damage_location': 'right_rear_wing', 'score': 0.335132}, {'bbox': [663, 322, 912, 460], 'damage_category': 'severe_scratch', 'damage_color': [0, 0, 120], 'damage_id': '2', 'damage_location': 'rear_bumper', 'score': 0.531722}, {'bbox': [866, 226, 923, 296], 'damage_category': 'severe_scratch', 'damage_color': [0, 0, 120], 'damage_id': '2', 'damage_location': 'right_rear_wing', 'score': 
0.504276}, {'bbox': [819, 540, 902, 596], 'damage_category': 'severe_scratch', 'damage_color': [0, 0, 120], 'damage_id': '2', 'damage_location': 'rear_bumper', 'score': 0.347227}, {'bbox': [689, 577, 796, 603], 'damage_category': 'severe_scratch', 'damage_color': [0, 0, 120], 'damage_id': '2', 'damage_location': 'rear_bumper', 'score': 0.326334}, {'bbox': [681, 542, 914, 610], 'damage_category': 'severe_scratch', 'damage_color': [0, 0, 120], 'damage_id': '2', 'damage_location': 'rear_bumper', 'score': 0.326196}, {'bbox': [295, 589, 866, 710], 'damage_category': 'medium_deformation', 'damage_color': [0, 115, 255], 'damage_id': '4', 'damage_location': 'rear_bumper', 'score': 0.792776}, {'bbox': [900, 13, 1030, 236], 'damage_category': 'medium_deformation', 'damage_color': [0, 115, 255], 'damage_id': '4', 'damage_location': 'right_rear_wing', 'score': 0.589383}, {'bbox': [908, 141, 1032, 279], 'damage_category': 'fender/bumper_damage', 'damage_color': [70, 50, 0], 'damage_id': '8', 'damage_location': 'right_rear_wing', 'score': 0.507765}]}, 'output_url': 'https://vpai-demo-portal-shanghai.oss-accelerate.aliyuncs.com/vehicle-damage%2Fdraw-020216.3770_20220626020216.jpg?OSSAccessKeyId=LTAI5tGgUWzx5gZopTcAvNtp&Expires=1656208996&Signature=IvLWGlP4W9EuiP75u44NQHUnVmY%3D', 'url_expiry': '2022/06/26 02:03:16'}, {'draw_result': True, 'job_id': 'f3ced372-9474-4428-ab0c-72536532db9d', 'output': {'elements': [{'bbox': [307, 604, 457, 710], 'damage_category': 'slight_scratch', 'damage_color': [50, 50, 100], 'damage_id': '1', 'damage_location': 'rear_bumper', 'score': 0.497312}, {'bbox': [963, 77, 1030, 155], 'damage_category': 'slight_scratch', 'damage_color': [50, 50, 100], 'damage_id': '1', 'damage_location': 'right_front_door', 'score': 0.469545}, {'bbox': [306, 653, 366, 694], 'damage_category': 'slight_scratch', 'damage_color': [50, 50, 100], 'damage_id': '1', 'damage_location': 'rear_bumper', 'score': 0.403235}, {'bbox': [405, 609, 460, 711], 'damage_category': 'slight_scratch', 'damage_color': [50, 50, 100], 'damage_id': '1', 'damage_location': 'rear_bumper', 'score': 0.332449}, {'bbox': [434, 14, 553, 144], 'damage_category': 'severe_scratch', 'damage_color': [0, 0, 120], 'damage_id': '2', 'damage_location': 'right_rear_wing', 'score': 0.698856}, {'bbox': [12, 437, 152, 508], 'damage_category': 'severe_scratch', 'damage_color': [0, 0, 120], 'damage_id': '2', 'damage_location': 'rear_bumper', 'score': 0.574816}, {'bbox': [7, 438, 201, 714], 'damage_category': 'severe_scratch', 'damage_color': [0, 0, 
120], 'damage_id': '2', 'damage_location': 'rear_bumper', 'score': 0.54255}, {'bbox': [814, 6, 853, 59], 'damage_category': 'severe_scratch', 'damage_color': [0, 0, 120], 'damage_id': '2', 'damage_location': 'right_rear_door', 'score': 0.492909}, {'bbox': [1058, 4, 1123, 81], 'damage_category': 'severe_scratch', 'damage_color': [0, 0, 120], 'damage_id': '2', 'damage_location': 'right_front_tire', 'score': 0.464681}, {'bbox': [437, 13, 519, 62], 'damage_category': 'severe_scratch', 'damage_color': [0, 0, 120], 'damage_id': '2', 'damage_location': 'right_rear_wing', 'score': 0.428339}, {'bbox': [277, 237, 391, 285], 'damage_category': 'severe_scratch', 'damage_color': [0, 0, 120], 'damage_id': '2', 'damage_location': 'right_rear_wing', 'score': 0.373748}, {'bbox': [52, 625, 116, 699], 'damage_category': 'severe_scratch', 'damage_color': [0, 0, 120], 'damage_id': '2', 'damage_location': 'rear_bumper', 'score': 0.330724}, {'bbox': [453, 232, 613, 388], 'damage_category': 'severe_scratch', 'damage_color': [0, 0, 120], 'damage_id': '2', 'damage_location': 'right_rear_tire', 'score': 0.328965}, {'bbox': [772, 0, 950, 
289], 'damage_category': 'medium_deformation', 'damage_color': [0, 115, 255], 'damage_id': '4', 'damage_location': 'right_rear_door', 'score': 0.399255}, {'bbox': [953, 0, 1127, 186], 'damage_category': 'medium_deformation', 'damage_color': [0, 115, 255], 'damage_id': '4', 'damage_location': 'right_front_door', 'score': 0.359885}, {'bbox': [155, 0, 709, 492], 'damage_category': 'severe_deformation', 'damage_color': [0, 
95, 255], 'damage_id': '5', 'damage_location': 'right_rear_wing', 'score': 0.718304}, {'bbox': [753, 2, 960, 294], 'damage_category': 'severe_deformation', 'damage_color': [0, 95, 255], 'damage_id': '5', 'damage_location': 'right_rear_door', 'score': 0.532058}, {'bbox': [126, 377, 
504, 489], 'damage_category': 'fender/bumper_damage', 'damage_color': [70, 50, 0], 'damage_id': '8', 'damage_location': 'rear_bumper', 'score': 0.945947}]}, 'output_url': 'https://vpai-demo-portal-shanghai.oss-accelerate.aliyuncs.com/vehicle-damage%2Fdraw-020220.7724_20220626020220.jpg?OSSAccessKeyId=LTAI5tGgUWzx5gZopTcAvNtp&Expires=1656209001&Signature=%2BLMZtSzsAuWuEQ4ywxn8bseEFSU%3D', 'url_expiry': '2022/06/26 02:03:21'}, {'draw_result': True, 'job_id': '660a1fe5-d3ed-4a9e-8574-ecbdadbea5d2', 'output': {'elements': [{'bbox': [11, 372, 81, 481], 'damage_category': 'slight_scratch', 'damage_color': [50, 50, 100], 'damage_id': '1', 'damage_location': 'right_rear_wing', 'score': 0.565272}, {'bbox': [505, 203, 561, 283], 'damage_category': 'severe_scratch', 'damage_color': [0, 0, 120], 'damage_id': '2', 'damage_location': 'right_rear_door', 'score': 0.773439}, {'bbox': [921, 165, 974, 220], 'damage_category': 'severe_scratch', 'damage_color': [0, 0, 120], 'damage_id': '2', 'damage_location': 'right_front_door', 'score': 0.564497}, {'bbox': [729, 211, 765, 256], 'damage_category': 'severe_scratch', 'damage_color': [0, 0, 120], 'damage_id': '2', 'damage_location': 'right_front_door', 'score': 0.386652}, {'bbox': [1063, 264, 1161, 360], 'damage_category': 'severe_scratch', 'damage_color': [0, 0, 120], 'damage_id': '2', 'damage_location': 'right_front_wing', 'score': 0.380715}, {'bbox': [0, 264, 230, 494], 'damage_category': 'medium_deformation', 'damage_color': [0, 115, 255], 'damage_id': '4', 'damage_location': 'right_rear_wing', 'score': 0.675253}, {'bbox': [429, 225, 760, 501], 'damage_category': 'medium_deformation', 'damage_color': [0, 115, 255], 'damage_id': '4', 'damage_location': 'right_rear_door', 'score': 0.326208}, {'bbox': [796, 93, 1193, 445], 'damage_category': 'severe_deformation', 'damage_color': [0, 95, 255], 'damage_id': '5', 'damage_location': 'right_front_door', 'score': 0.948401}, {'bbox': [371, 122, 796, 539], 'damage_category': 'severe_deformation', 'damage_color': [0, 95, 255], 'damage_id': '5', 'damage_location': 'right_rear_door', 'score': 0.691995}, {'bbox': [1108, 354, 1159, 397], 'damage_category': 'crack_and_hole', 'damage_color': [75, 100, 0], 'damage_id': '6', 'damage_location': 'right_front_wing', 'score': 0.477902}]}, 'output_url': 'https://vpai-demo-portal-shanghai.oss-accelerate.aliyuncs.com/vehicle-damage%2Fdraw-020225.1322_20220626020225.jpg?OSSAccessKeyId=LTAI5tGgUWzx5gZopTcAvNtp&Expires=1656209005&Signature=Wlhvq2aOgbTgo5ORlxt5zgIkK48%3D', 'url_expiry': '2022/06/26 02:03:25'}, {'draw_result': True, 'job_id': '4b6f5400-2fa0-4990-9ab4-501cf2c58b2d', 'output': {'elements': [{'bbox': [37, 318, 209, 373], 'damage_category': 'slight_scratch', 'damage_color': [50, 50, 100], 'damage_id': '1', 'damage_location': 'right_rear_door', 'score': 0.373566}, {'bbox': [328, 283, 390, 325], 'damage_category': 'slight_scratch', 'damage_color': [50, 50, 100], 'damage_id': '1', 'damage_location': 'right_rear_door', 'score': 0.333977}, {'bbox': [611, 176, 711, 256], 'damage_category': 'severe_scratch', 'damage_color': [0, 0, 120], 'damage_id': '2', 'damage_location': 'right_front_door', 'score': 0.853403}, {'bbox': [1067, 133, 1130, 225], 'damage_category': 'severe_scratch', 'damage_color': [0, 0, 120], 'damage_id': '2', 'damage_location': 'right_front_wing', 'score': 0.502186}, {'bbox': [237, 280, 315, 349], 'damage_category': 'severe_scratch', 'damage_color': [0, 0, 120], 'damage_id': '2', 'damage_location': 'right_rear_door', 'score': 0.435874}, 
{'bbox': [1, 278, 361, 647], 'damage_category': 'medium_deformation', 'damage_color': [0, 115, 255], 'damage_id': '4', 'damage_location': 'right_rear_door', 'score': 0.705571}, {'bbox': [364, 41, 1197, 577], 'damage_category': 'severe_deformation', 'damage_color': [0, 95, 255], 'damage_id': '5', 'damage_location': 'right_front_door', 'score': 0.981817}, {'bbox': [1011, 468, 1050, 510], 'damage_category': 'crack_and_hole', 'damage_color': [75, 100, 0], 'damage_id': '6', 'damage_location': 'right_front_wing', 'score': 0.319893}]}, 'output_url': 'https://vpai-demo-portal-shanghai.oss-accelerate.aliyuncs.com/vehicle-damage%2Fdraw-020229.1807_20220626020229.jpg?OSSAccessKeyId=LTAI5tGgUWzx5gZopTcAvNtp&Expires=1656209009&Signature=EqKHV%2BujWSmzKX0ikAutGkITM4I%3D', 'url_expiry': '2022/06/26 02:03:29'}, {'draw_result': True, 'job_id': '06835275-6d1e-4eb8-917d-c30a65a16c77', 'output': {'elements': [{'bbox': [826, 318, 976, 380], 'damage_category': 'slight_scratch', 'damage_color': 
[50, 50, 100], 'damage_id': '1', 'damage_location': 'front_bumper', 'score': 0.50894}, {'bbox': [936, 608, 1019, 698], 'damage_category': 'slight_scratch', 'damage_color': [50, 50, 100], 'damage_id': '1', 'damage_location': 'front_bumper', 'score': 0.320937}, {'bbox': [576, 368, 739, 424], 'damage_category': 'severe_scratch', 'damage_color': [0, 0, 120], 'damage_id': '2', 'damage_location': 'right_front_wing', 'score': 
0.782618}, {'bbox': [480, 454, 585, 512], 'damage_category': 'severe_scratch', 'damage_color': [0, 0, 120], 'damage_id': '2', 'damage_location': 'right_front_wing', 'score': 0.560554}, {'bbox': [1036, 434, 1158, 640], 'damage_category': 'severe_scratch', 'damage_color': [0, 0, 120], 'damage_id': '2', 'damage_location': 'front_bumper', 'score': 0.378829}, {'bbox': [917, 330, 1169, 696], 'damage_category': 'medium_deformation', 'damage_color': [0, 115, 255], 'damage_id': '4', 'damage_location': 'front_bumper', 'score': 0.68045}, {'bbox': [179, 276, 960, 602], 
'damage_category': 'medium_deformation', 'damage_color': [0, 115, 255], 'damage_id': '4', 'damage_location': 'right_front_wing', 'score': 0.533388}, {'bbox': [153, 253, 972, 636], 'damage_category': 'severe_deformation', 'damage_color': [0, 95, 255], 'damage_id': '5', 'damage_location': 'right_front_wing', 'score': 0.484418}, {'bbox': [823, 614, 915, 699], 'damage_category': 'crack_and_hole', 'damage_color': [75, 100, 0], 'damage_id': '6', 'damage_location': 'front_bumper', 'score': 0.332555}, {'bbox': [794, 313, 1045, 643], 'damage_category': 'fender/bumper_damage', 'damage_color': [70, 50, 0], 'damage_id': '8', 'damage_location': 'front_bumper', 'score': 0.898714}]}, 'output_url': 'https://vpai-demo-portal-shanghai.oss-accelerate.aliyuncs.com/vehicle-damage%2Fdraw-020232.9847_20220626020233.jpg?OSSAccessKeyId=LTAI5tGgUWzx5gZopTcAvNtp&Expires=1656209013&Signature=lKD74Um0gP%2BitEf0tSub%2Fx3GMhE%3D', 'url_expiry': '2022/06/26 02:03:33'}]

In [ ]:
from collections import defaultdict
import copy
def calculate_recovery(cost_mapping, damage_details=[]):







  

  recovery = defaultdict(lambda :0)
  cost_mapping_mine = defaultdict(lambda : 2000)
  for d in cost_mapping:
    cost_mapping_mine[d] = cost_mapping[d]
  
  weighing_factor = defaultdict(lambda : 1)
  for it, dd in enumerate(damage_details):
    entities_damaged = []
    
    for rec in dd["output"]["elements"] :
      entities_damaged.append((rec["damage_location"], rec["score"]))
    
    present_addition = defaultdict(lambda x: 0)
    if it != 0:
      store_recovery = copy.deepcopy(recovery)
    
    for damage_part, damage_percent in entities_damaged:
      present_addition[damage_part] = cost_mapping_mine[damage_part]*damage_percent
      recovery[damage_part.replace("_", " ")] += cost_mapping_mine[damage_part]*damage_percent
    
    
    
    if it == 0:
      prev_entities = [ ed[0] for ed in entities_damaged]
    else:
      present_entities = [ ed[0] for ed in entities_damaged]
      for ent in set(present_entities):
        if ent in set(prev_entities):
          weighing_factor[ent] += 1
        else:
          weighing_factor[ent] = 1
      prev_entities = present_entities 

      
      already_have = []
      
      for damage_part, damage_percent in entities_damaged:
        if not(damage_part in already_have):
          recovery[damage_part.replace("_", " ")] = (store_recovery[damage_part.replace("_", " ")]*(1 if weighing_factor[damage_part]-1 == 0 else weighing_factor[damage_part]-1)+ present_addition[damage_part])/weighing_factor[damage_part]
          already_have.append(damage_part)
          
        
      
  total_sum = 0
  for i in recovery.values():
    total_sum += i #recovery.values()
  recovery = dict(recovery)
  for k in recovery:
    recovery[k] = int(recovery[k])
  
  recovery['sum'] = int(total_sum)
  return recovery

calculate_recovery(cost_mapping, arg)


{'front bumper': 1797,
 'rear bumper': 2198,
 'right front door': 1526,
 'right front tire': 929,
 'right front wing': 854,
 'right rear door': 1286,
 'right rear tire': 657,
 'right rear wing': 1267,
 'right tail light': 813,
 'sum': 11332}

In [ ]:
re.findall("/watch\?v=(.*)&list", "https://www.youtube.com/watch?v=fAiL1sgCUIk&list=PLXSx78snmRzmSEdoCFUkna_E0YHwZzZgb&index=49")

In [ ]:
bot.chat("what health plans are available for women")

In [ ]:
bot.chat("health plans for women")

In [ ]:
bot.vsearch_engine.search_snip("when does avatar 2 release")
bot.rsearch_engine.search_snip("when does avatar 2 release")
bot.stsearch_engine.search_snip("when does avatar 2 release")

[('Sorry no appropriate response could solve your query try framing a different query',
  10000000.0)]

In [ ]:
bot.recommend_chat_queries(1)

In [ ]:
import pandas as pd

In [ ]:
%cd ./resonancebrains
!vim views.py

In [ ]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(8000)"))

In [ ]:
!python manage.py migrate
!python manage.py runserver 8000

In [ ]:
class ClaimsSearch(FAISS):
  
  def __init__(self, semantic_outlier_radius):
    self.encoder = USE(model_path)
    self.d = self.encoder.encode(['hello']).shape[-1]
    super().__init__(self.d)
    self.semantic_outlier_radius = semantic_outlier_radius
    
    
  def add(self, v, response,):
    self.index.add(v)
    snip_details = (response, v)
    self.snips[self.snip_id] = snip_details
    self.snip_id += 1
  
  def add_pool(self, cmds=[]):
    
    for c in tqdm(cmds):
        
        text = c[0].lower() 
        #res = self.conversor(transformers.Conversation(q.lower()), pad_token_id=50256)
        #res = str(res)
        #res = res[res.find("bot >> ")+6:].strip()
        emb = self.encoder.encode(text)
        self.add(emb, c[1])
    return 1
  
  def search_snip(self, s, k=10):

      emb = self.encoder.encode(s)
      res = self.search(emb, k)
      res = [r for r in res if r[-1]<=self.semantic_outlier_radius]  #recommendations in radius of 100 euclideans
      if len(res) == 0:
        return [("Sorry no appropriate response could solve your query try framing a different query", 1e+5)]
      return res



def get_claim_action(usr_inp, clsearch_engine):
  return clsearch_engine.search_snip(usr_inp, 1)[0][0]

clsearch_engine = ClaimsSearch(1)
clsearch_engine.add_pool(cmds=[('property claims', {'mobile': False, 'motor': False, 'property': True}),
('vehicle claims', {'mobile': False, 'motor': True, 'property': False}),
('motor claims', {'mobile': False, 'motor': True, 'property': False}),
('mobile claims', {'mobile': False, 'motor': True, 'property': False}),
('cell phone claims', {'mobile': True, 'motor': False, 'property': False}),
('bike claim', {'mobile': False, 'motor': True, 'property': False}),
('car claim', {'mobile': False, 'motor': True, 'property': False}),
('apply for property insurance',
  {'mobile': False, 'motor': False, 'property': True}),
('apply for vehicle insurance',
  {'mobile': False, 'motor': True, 'property': False}),
('apply for motor insurance',
  {'mobile': False, 'motor': True, 'property': False}),
('apply for car insurance',
  {'mobile': False, 'motor': True, 'property': False}),
('apply for bike insurance',
  {'mobile': False, 'motor': True, 'property': False}),
('apply for mobile insurance',
  {'mobile': True, 'motor': False, 'property': False}),
('apply for cell phone insurance',
  {'mobile': True, 'motor': False, 'property': False}),
('apply for phone insurance',
  {'mobile': True, 'motor': False, 'property': False})])
get_claim_action("I want vehicle insurance", clsearch_engine)

100%|██████████| 15/15 [00:00<00:00, 36.29it/s]


{'mobile': False, 'motor': True, 'property': False}

In [ ]:
get_claim_action("My phone is cracked can i get insurance", clsearch_engine)

{'mobile': True, 'motor': False, 'property': False}

In [ ]:
small_talk_dataset = """<ol>
<li>Hi / Hello / How are you?</li>
<li>What’s up? / Yo / How you doin?</li>
<li>Good Morning / Good Evening / Good Afternoon / Good Night</li>
<li>What can you do? / What can I do with you? / What are you doing?</li>
<li>Thanks / Thank you / Thanks dude / Thanks lady</li>
<li>My name is &lt;insert user name&gt;</li>
<li>How can you help me? / Help me please</li>
<li>Talk to you later!</li>
<li>Tell me something that you can do</li>
<li>Say anything</li>
</ol>

<ol>
<li>Will you marry me? / Are you Single? / Do you have a boyfriend/girlfriend?</li><span class="ezoic-autoinsert-ad ezoic-long_content"></span><span style="clear:both; display:block" id="ez-clearholder-banner-1"></span><span class="ezoic-ad banner-1 banner-1360 adtester-container adtester-container-360" data-ez-name="chatbotbusinessframework_com-banner-1" style="min-height: 250px;"><span id="div-gpt-ad-chatbotbusinessframework_com-banner-1-0" ezaw="300" ezah="250" style="position: relative; z-index: 0; padding: 0px; min-height: 250px; min-width: 300px;" class="ezoic-ad ezfound" data-google-query-id="CIuoy_S7vvgCFQmS2AUdmkwEzw"><div id="google_ads_iframe_/1254144,22621963351/chatbotbusinessframework_com-banner-1_0__container__" style="border: 0pt none; display: inline-block; width: 300px; height: 250px;"><iframe frameborder="0" src="https://6ea3822cf5edd23b3b9e97e6a2c33c20.safeframe.googlesyndication.com/safeframe/1-0-38/html/container.html" id="google_ads_iframe_/1254144,22621963351/chatbotbusinessframework_com-banner-1_0" title="3rd party ad content" name="" scrolling="no" marginwidth="0" marginheight="0" width="300" height="250" data-is-safeframe="true" sandbox="allow-forms allow-popups allow-popups-to-escape-sandbox allow-same-origin allow-scripts allow-top-navigation-by-user-activation" role="region" aria-label="Advertisement" tabindex="0" data-google-container-id="7" style="border: 0px; vertical-align: bottom;" data-load-complete="true"></iframe></div></span></span>
<li>Do you know other chatbots?</li>
<li>Do you want to rule the world?</li>
<li>Are you busy?</li>
<li>Say something funny</li>
<li>Can you say anything else</li>
<li>What do you think of me?</li>
<li>Happy &lt;insert holiday&gt;</li>
<li>Do you love me?</li>
<li>Can you fly?</li>
</ol>

<ol>
<li>You are a genius!</li>
<li>You are awesome / great / special</li>
<li>Thanks so much!</li><span class="ezoic-autoinsert-ad ezoic-longer_content"></span><span style="clear:both; display:block" id="ez-clearholder-large-leaderboard-2"></span><span class="ezoic-ad large-leaderboard-2 large-leaderboard-2550 adtester-container adtester-container-550" data-ez-name="chatbotbusinessframework_com-large-leaderboard-2" style="min-height: 250px;"><span id="div-gpt-ad-chatbotbusinessframework_com-large-leaderboard-2-0" ezaw="300" ezah="250" style="position: relative; z-index: 0; padding: 0px; min-height: 250px; min-width: 300px;" class="ezoic-ad ezfound"><div id="google_ads_iframe_/1254144,22621963351/chatbotbusinessframework_com-large-leaderboard-2_0__container__" style="border: 0pt none; display: inline-block;"></div></span></span>
<li>Thanks, that’s very helpful</li>
<li>Yup, that’s true</li>
<li>That’s accurate</li>
<li>You are right</li>
<li>Give me a fist bump</li>
<li>I think you are great!</li>
<li>Thumbs up</li>
</ol>

<ol>
<li>You suck</li>
<li>I hate you</li>
<li>Go away</li>
<li>That’s not right</li>
<li>That’s not true</li><span class="ezoic-autoinsert-ad ezoic-longest_content"></span><span style="clear:both; display:block" id="ez-clearholder-leader-1"></span><span class="ezoic-ad leader-1 leader-1560 adtester-container adtester-container-560" data-ez-name="chatbotbusinessframework_com-leader-1" style="min-height: 60px;"><span id="div-gpt-ad-chatbotbusinessframework_com-leader-1-0" ezaw="336" ezah="280" style="position: relative; z-index: 0; padding: 0px; width: 100%; max-width: 1200px; min-height: 60px; min-width: 468px; margin-left: auto !important; margin-right: auto !important;" class="ezoic-ad ezfound" data-google-query-id="COS_vdG7vvgCFWua2AUdUnUKow"><div id="google_ads_iframe_/1254144,22621963351/chatbotbusinessframework_com-leader-1_0__container__" style="border: 0pt none; display: inline-block; width: 468px; height: 60px;"><iframe frameborder="0" src="https://6ea3822cf5edd23b3b9e97e6a2c33c20.safeframe.googlesyndication.com/safeframe/1-0-38/html/container.html" id="google_ads_iframe_/1254144,22621963351/chatbotbusinessframework_com-leader-1_0" title="3rd party ad content" name="" scrolling="no" marginwidth="0" marginheight="0" width="468" height="60" data-is-safeframe="true" sandbox="allow-forms allow-popups allow-popups-to-escape-sandbox allow-same-origin allow-scripts allow-top-navigation-by-user-activation" role="region" aria-label="Advertisement" tabindex="0" data-google-container-id="9" style="border: 0px; vertical-align: bottom;" data-load-complete="true"></iframe></div></span></span>
<li>That was stupid</li>
<li>You are not helpful</li>
<li>That was not the right answer</li>
<li>Terrible</li>
<li>&lt;expletive&gt;</li>
</ol>

<ol>
<li>Which baseball / football team do you like?</li>
<li>You know nothing Jon Snow.</li>
<li>Where’s Waldo?</li>
<li>What’s the weather?</li>
<li>Where’s the best restaurant in town?</li>
<li>What should I eat today?</li>
<li>Who is the best &lt;insert role&gt;?</li><span class="ezoic-autoinsert-ad ezoic-incontent_5"></span><span style="clear:both; display:block" id="ez-clearholder-large-mobile-banner-1"></span><span class="ezoic-ad large-mobile-banner-1 large-mobile-banner-1700 adtester-container adtester-container-700" data-ez-name="chatbotbusinessframework_com-large-mobile-banner-1" style="min-height: 250px;"><span id="div-gpt-ad-chatbotbusinessframework_com-large-mobile-banner-1-0" ezaw="300" ezah="250" style="position: relative; z-index: 0; padding: 0px; min-height: 250px; min-width: 300px;" class="ezoic-ad ezfound" data-google-query-id="CIDt4tK7vvgCFQ3TcwEdbZgHCA"><div id="google_ads_iframe_/1254144,22621963351/chatbotbusinessframework_com-large-mobile-banner-1_0__container__" style="border: 0pt none; display: inline-block; width: 300px; height: 250px;"><iframe frameborder="0" src="https://6ea3822cf5edd23b3b9e97e6a2c33c20.safeframe.googlesyndication.com/safeframe/1-0-38/html/container.html" id="google_ads_iframe_/1254144,22621963351/chatbotbusinessframework_com-large-mobile-banner-1_0" title="3rd party ad content" name="" scrolling="no" marginwidth="0" marginheight="0" width="300" height="250" data-is-safeframe="true" sandbox="allow-forms allow-popups allow-popups-to-escape-sandbox allow-same-origin allow-scripts allow-top-navigation-by-user-activation" role="region" aria-label="Advertisement" tabindex="0" data-google-container-id="a" style="border: 0px; vertical-align: bottom;" data-load-complete="true"></iframe></div></span></span>
<li>Who should I vote for?</li>
<li>Who is the best digital assistant?</li>
<li>Do you know Alexa?</li>
</ol>

<ol>
<li>What’s your age?</li>
<li>Can you sleep?</li>
<li>Who is your boss?</li>
<li>Who created you?</li>
<li>Do you know other chatbots?</li>
<li>What is your favorite color?</li>
<li>Why did the chicken cross the road?</li>
<li>Happy Birthday</li>
<li>Do you know me?</li>
<li>Are you my assistant?</li>
</ol>"""

import re
matches = re.findall("<li>(.*)</li>", small_talk_dataset)
small_talk = []
for m in matches:
  l = [s.strip() for s in m.split("/")]
  small_talk += l



In [ ]:
bot = ChatBot()
bot.chat("hello")

100%|██████████| 79/79 [00:01<00:00, 61.72it/s]


{'response': 'Hello! :D', 'status': 1}

In [ ]:
#property claim = 0, vehicle/motor/bike/car claim = 1, mobile/cell phone/phone claim=2, 
dic = ["property","motor", "mobile"]
claim_cmds = [("property claims", 0), ("vehicle claims", 1), ("motor claims", 1), ("mobile claims", 1), ("cell phone claims", 2), ("bike claim", 1), ("car claim", 1), ("apply for property insurance", 0), ("apply for vehicle insurance", 1), ("apply for motor insurance", 1), ("apply for car insurance", 1), ("apply for bike insurance", 1), ("apply for mobile insurance", 2), ("apply for cell phone insurance", 2), ("apply for phone insurance", 2)]
claim_cmdst = []
for c in claim_cmds:
  dicc = {"property":False, "motor":False,"mobile":False}
  dicc[dic[c[1]]] = True

  claim_cmdst.append((c[0], dicc))
claim_cmdst

[('property claims', {'mobile': False, 'motor': False, 'property': True}),
 ('vehicle claims', {'mobile': False, 'motor': True, 'property': False}),
 ('motor claims', {'mobile': False, 'motor': True, 'property': False}),
 ('mobile claims', {'mobile': False, 'motor': True, 'property': False}),
 ('cell phone claims', {'mobile': True, 'motor': False, 'property': False}),
 ('bike claim', {'mobile': False, 'motor': True, 'property': False}),
 ('car claim', {'mobile': False, 'motor': True, 'property': False}),
 ('apply for property insurance',
  {'mobile': False, 'motor': False, 'property': True}),
 ('apply for vehicle insurance',
  {'mobile': False, 'motor': True, 'property': False}),
 ('apply for motor insurance',
  {'mobile': False, 'motor': True, 'property': False}),
 ('apply for car insurance',
  {'mobile': False, 'motor': True, 'property': False}),
 ('apply for bike insurance',
  {'mobile': False, 'motor': True, 'property': False}),
 ('apply for mobile insurance',
  {'mobile': True, 'm

In [ ]:
import gradio as gr
bot = ChatBot(small_talk)
chatbot = gr.Chatbot()
demo = gr.Interface(
    bot.chat,
    ["text", "state"],
    [chatbot, "state"],
    allow_screenshot=False,
    allow_flagging="never",
)
demo.launch()

#gr.Chatbot(fn=bot.chat, value).launch()

100%|██████████| 79/79 [00:01<00:00, 51.63it/s]
/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: `allow_screenshot` parameter is deprecated, and it has no effect
  warnings.warn(value)


Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://41999.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<gradio.routes.App at 0x7effa49068d0>,
 'http://127.0.0.1:7860/',
 'https://41999.gradio.app')

In [ ]:
bot.vsearch_engine.search_snip("what is an upi", 1), bot.rsearch_engine.search_snip("explain emi calculators", 1)

([('https://www.youtube.com/watch?v=N2p4ZC7XkdU&list=PLXSx78snmRzmzkiBBiw-l0oeXtz4_RKP3&index=6',
   'what is gst',
   0.98499596)],
 [('This doctor loan calculator is simple to use. Enter the total loan amount, the rate of interest and your chosen tenor in months. The calculator will instantly show the EMIs based on the details you provided.',
   ' doctor-loan-emi-calculator',
   0.8894903)])

In [ ]:
bot.vsearch_engine.search_snip("why do I need maternity insurance")

[('https://www.youtube.com/watch?v=j0fLRSYA3KQ&list=PLXSx78snmRzlE_LBV-yVwBsRSk1nUut3W&index=15',
  'how to renew your health insurance',
  0.99353546)]

In [ ]:

bot.chat("what is love","")

([('what is love',
   'Redemption is when investors choose to withdraw their investments at the applicable NAV. The liquidity of mutual fund investment refers to redemption. It can be done either online or offline. A redemption fee is also charged in this process.')],
 [('what is love',
   'Redemption is when investors choose to withdraw their investments at the applicable NAV. The liquidity of mutual fund investment refers to redemption. It can be done either online or offline. A redemption fee is also charged in this process.')])

In [ ]:
#st = SmallTalkSearch(10).
#st.add_pool(small_talk)
st.search_snip("what is credit card")

[('what’s up?', 1.3084565),
 ('what’s the weather?', 1.3198122),
 ('which baseball', 1.4258513),
 ('what’s your age?', 1.4340068),
 ('who is your boss?', 1.4744143),
 ('who created you?', 1.5477237),
 ('what do you think of me?', 1.5523068),
 ('why did the chicken cross the road?', 1.5728647),
 ('who is the best digital assistant?', 1.5748867),
 ('where’s the best restaurant in town?', 1.5985329)]

In [ ]:
rs = StaticQuerySearch(10)
rs.add_pool([])

In [ ]:
import random

import gradio as gr


def chat(message, history):
    history = history or []
    if message.startswith("How many"):
        response = random.randint(1, 10)
    elif message.startswith("How"):
        response = random.choice(["Great", "Good", "Okay", "Bad"])
    elif message.startswith("Where"):
        response = random.choice(["Here", "There", "Somewhere"])
    else:
        response = "I don't know"
    history.append((message, response))
    return history, history

chatbot = gr.Chatbot()
demo = gr.Interface(
    chat,
    ["text", "state"],
    [chatbot, "state"],
    allow_screenshot=False,
    allow_flagging="never",
)
demo.launch()

/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: `allow_screenshot` parameter is deprecated, and it has no effect
  warnings.warn(value)


Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://14464.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<gradio.routes.App at 0x7f8a2b518f50>,
 'http://127.0.0.1:7868/',
 'https://14464.gradio.app')

In [ ]:
vsearch_engine = VideoSemanticSearch()
vsearch_engine.add_pool(filepaths=['/content/keywords_timestamps.txt', '/content/transcript_timestamps.txt'])
vsearch_engine.search_snip('policy claim', 1)  #gives <matching_doc_term>, <doc_id>, <term_timestamps:(start, end)>

2it [00:02,  1.37s/it]


[('policy services', 0, [{'start': 76052, 'end': 76950}], 0.6322691)]

In [ ]:
vsearch_engine.search_snip('who should i appraoch for insurance guidance', 1)

[('out to our insurance consultant or',
  1,
  [{'start': 105.09, 'end': 109.649}],
  0.95624423)]

In [ ]:
rsearch_engine = StaticQuerySearch(10)
rsearch_engine.add_pool(filepaths=['/content/FINAL DB.xlsx'])
rsearch_engine.search_snip('what is love?', 1) #gives <relevant_response>, <intent>

100%|██████████| 1/1 [00:17<00:00, 17.87s/it]


[('Redemption is when investors choose to withdraw their investments at the applicable NAV. The liquidity of mutual fund investment refers to redemption. It can be done either online or offline. A redemption fee is also charged in this process.',
  ' mutual-funds-faq',
  1.0007002)]

In [ ]:
rsearch_engine.search_snip('what is credit card', 10) #

[("A credit card is a payment instrument that gives customers a pre-set credit limit. Using this credit limit, the customer can pay for purchases without having to pay in cash or issuing a cheque. The card’s limit is decided by the financial institution depending on the customer's credit score and monthly income.",
  ' credit-card-eligibility-and-documents',
  0.05928781),
 ('A credit card statement is issued by the bank or the credit card company at the end of the billing cycle. It presents a summary of all the transactions you have made during the billing period. The credit card statement mentions the amount you owe to the bank, the minimum amount due and the due date.',
  ' credit-card',
  0.26993),
 ('A credit card balance is the total amount of money you owe to the credit card company. It is mentioned in the credit card statement. You need to repay this amount on or before the due date to avoid any penalty.',
  ' credit-card',
  0.2923473),
 ('A credit card allows you to buy any p

In [ ]:
res = rsearch_engine.search_snip("how much emi can i get for 50 thouand at 5% rate of interest", k=1)
[_ for res in filter(lambda x: print(x[0]), res)]


In [ ]:
print("total video snips in database :", vsearch_engine.snip_id, "| total text-response snips in database :", rsearch_engine.snip_id)

total video snips in database : 53 | total text-response snips in database : 1063


**Chat Briefing Code Base**

In [ ]:
class ChatBriefing():
  def __init__(self):
    self.name = "Chat Briefer for User Chat Predominantly"

  def summarize(self, text, ratio=0.75):
        summarizer = pipeline("summarization", model="lidiya/bart-large-xsum-samsum")#"sshleifer/distilbart-cnn-12-6")
        summarized = summarizer(text, min_length=5, max_length=int(ratio*len(text.split(" ")))) #75% summarization by default

        return summarized

In [ ]:
user_conversation = """what do you need?, 
I need student oriented policy 
How to apply for a student oriented policy
do you want me to wlk through form filling
Walk me through the form filling
user name : Praneeth Thota
Password : ***$&&%4$%
want it done today itself 
how was the service
It was a great service, but would like to talk with executive I have some concerns about health to raise"""

briefing_engine = ChatBriefing()
summarized = briefing_engine.summarize(user_conversation, ratio=1.0)
print(summarized[0]['summary_text'])

Praneeth Thota needs to apply for a student oriented policy. He would like to talk to executive about his concerns about health.


### Ignore

In [ ]:
!pip install pyttsx3
!sudo apt install libespeak1

In [ ]:
# Python program to show
# how to convert text to speech
import pyttsx3

# Initialize the converter
converter = pyttsx3.init()

# Set properties before adding
# Things to say

# Sets speed percent
# Can be more than 100
converter.setProperty('rate', 150)
# Set volume 0-1
converter.setProperty('volume', 0.7)

# Queue the entered text
# There will be a pause between
# each one like a pause in
# a sentence
converter.say("Hello GeeksforGeeks")
converter.say("I'm also a geek")

# Empties the say() queue
# Program will not continue
# until all speech is done talking
converter.runAndWait()


In [ ]:
import speech_recognition as sr
# Beginning of the AI
class ChatBot():
    def __init__(self, name):
      print("----- starting up", name, "-----")
      self.name = name
    def speech_to_text(self):
      recognizer = sr.Recognizer()
      with sr.Microphone() as mic:
          print("listening...")
          audio = recognizer.listen(mic)
      try:
          self.text = recognizer.recognize_google(audio)
          print("me --> ", self.text)
      except:
          print("me -->  ERROR")
# Execute the AI
if __name__ == "__main__":
    ai = ChatBot(name="filos")



----- starting up filos -----


In [ ]:
ai.speech_to_text()

In [ ]:
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libasound2-dev is already the newest version (1.1.3-5ubuntu0.6).
ffmpeg is already the newest version (7:3.4.11-0ubuntu0.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 184 kB of archives.
After this operation, 891 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudiocpp0 amd64 19.6.0-1 [15.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 portaudio19-dev amd64 19.6.0-1 [104 kB]
Fetched 184 kB in 0s (719 kB/s)
Selecting p

In [ ]:
!pip install pyaudio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pyaudio: filename=PyAudio-0.2.11-cp37-cp37m-linux_x86_64.whl size=52607 sha256=248d80f89e96716c4db2bb55882a3db177c38ff4b68880d255a0d0af4a1133e4
  Stored in directory: /root/.cache/pip/wheels/40/2e/4c/b71e7e96c861a46e6213bc6bb482b94dcf293a92c5e736c1ec
Successfully built pyaudio


In [ ]:
!pip install python_speech_features

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for python-speech-features: filename=python_speech_features-0.6-py3-none-any.whl size=5888 sha256=a2af3f00057a005db9a33afa8fe1d26daf444fc7bcbe6f9d383040a9d1a96dd7
  Stored in directory: /root/.cache/pip/wheels/b0/0e/94/28cd6afa3cd5998a63eef99fe31777acd7d758f59cf24839eb
Successfully built python-speech-features


In [ ]:
import tensorflow as tf
import numpy as np
import os
import glob
import pickle
import cv2
import time
import sys
from numpy import genfromtxt

from keras import backend as K
from keras.models import load_model
K.set_image_data_format('channels_first')
np.set_printoptions(threshold=sys.maxsize)

#np.set_printoptions(threshold=np.nan)


import pyaudio
from IPython.display import Audio, display, clear_output
import wave
from scipy.io.wavfile import read
#from sklearn.mixture import GMM 
from sklearn import mixture
GMM = mixture.GaussianMixture(n_components=3, covariance_type='full')
import warnings
warnings.filterwarnings("ignore")

from sklearn import preprocessing
import python_speech_features as mfcc

ModuleNotFoundError: ignored

In [ ]:
#Calculate and returns the delta of given feature vector matrix

def calculate_delta(array):
    rows,cols = array.shape
    deltas = np.zeros((rows,20))
    N = 2
    for i in range(rows):
        index = []
        j = 1
        while j <= N:
            if i-j < 0:
                first = 0
            else:
                first = i-j
            if i+j > rows -1:
                second = rows -1
            else:
                second = i+j
            index.append((second,first))
            j+=1
        deltas[i] = ( array[index[0][0]]-array[index[0][1]] + (2 * (array[index[1][0]]-array[index[1][1]])) ) / 10
    return deltas

#convert audio to mfcc features
def extract_features(audio,rate):    
    mfcc_feat = mfcc.mfcc(audio,rate, 0.025, 0.01,20,appendEnergy = True, nfft=1103)
    mfcc_feat = preprocessing.scale(mfcc_feat)
    delta = calculate_delta(mfcc_feat)

    #combining both mfcc features and delta
    combined = np.hstack((mfcc_feat,delta)) 
    return combined

def add_user():

    name = input("Enter Name:")
    #Voice authentication
    FORMAT = pyaudio.paInt16
    CHANNELS = 2
    RATE = 44100
    CHUNK = 1024
    RECORD_SECONDS = 3
    
    source = "/content/Voice-Authentication-and-Face-Recognition/voice_database/" + name
    
   
    os.mkdir(source)

    for i in range(3):
        audio = pyaudio.PyAudio()

        if i == 0:
            j = 3
            while j>=0:
                time.sleep(1.0)
                print("Speak your name in {} seconds".format(j))
                clear_output(wait=True)

                j-=1

        elif i ==1:
            print("Speak your name one more time")
            time.sleep(0.5)

        else:
            print("Speak your name one last time")
            time.sleep(0.5)

        # start Recording
        #stream = audio.open(format=FORMAT, channels=CHANNELS,
        #            rate=RATE, input=True,
        #            frames_per_buffer=CHUNK)

        #print("recording...")
        frames = record()#[]
        #print(frames)
        #for _ in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        #    data = stream.read(CHUNK)
        #    frames.append(data)

        # stop Recording
        #stream.stop_stream()
        #stream.close()
        audio.terminate()
        vector_bytes = frames
        vector_bytes_str = str(vector_bytes)
        vector_bytes_str_enc = vector_bytes_str.encode()
        bytes_np_dec = vector_bytes_str_enc.decode('unicode-escape').encode('ISO-8859-1')[2:-1]
        frames = bytes_np_dec
        # saving wav file of speaker
        waveFile = wave.open(source + '/' + str((i+1)) + '.wav', 'wb')
        waveFile.setnchannels(CHANNELS)
        waveFile.setsampwidth(audio.get_sample_size(FORMAT))
        waveFile.setframerate(RATE)
        waveFile.writeframes(frames)
        #waveFile.writeframes(b''.join(frames))
        waveFile.close()
        print("Done")

    dest =  "/content/Voice-Authentication-and-Face-Recognition/gmm_models/"
    count = 1

    for path in os.listdir(source):
        path = os.path.join(source, path)

        features = np.array([])
        
        # reading audio files of speaker
        (sr, audio) = read(path)
        
        # extract 40 dimensional MFCC & delta MFCC features
        vector   = extract_features(audio,sr)

        if features.size == 0:
            features = vector
        else:
            features = np.vstack((features, vector))
            
        # when features of 3 files of speaker are concatenated, then do model training
        if count == 3:    
            gmm = GMM(n_components = 16, n_iter = 200, covariance_type='diag',n_init = 3)
            gmm.fit(features)

            # saving the trained gaussian model
            pickle.dump(gmm, open(dest + name + '.gmm', 'wb'))
            print(name + ' added successfully') 
            
            features = np.asarray(())
            count = 0
        count = count + 1
def delete_user():
    name = input("Enter name of the user:")
    
    with open("./face_database/embeddings.pickle", "rb") as database:
        db = pickle.load(database)
        user = db.pop(name, None)
    
        if user is not None:
            print('User ' + name + ' deleted successfully')
            # save the database

            # remove the speaker wav files and gmm model
            [os.remove(path) for path in glob.glob('/content/Voice-Authentication-and-Face-Recognition/voice_database/' + name + '/*')]
            os.removedirs('./voice_database/' + name)
            os.remove('./gmm_models/' + name + '.gmm')
        
        else:
            print('No such user !!')

# all imports
from io import BytesIO
from base64 import b64decode
from google.colab import output
from IPython.display import Javascript

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=3):
  print("Speak Now...")
  display(Javascript(RECORD))
  sec += 1
  s = output.eval_js('record(%d)' % (sec*1000))
  print("Done Recording !")
  print(s)
  b = b64decode(s.split(',')[1])
  return b 

def recognize():
   # Voice Authentication
   FORMAT = pyaudio.paInt16
   CHANNELS = 2
   RATE = 44100
   CHUNK = 1024
   RECORD_SECONDS = 3
   FILENAME = "./test.wav"

   audio = pyaudio.PyAudio()
  
   # start Recording
   #stream = audio.open(format=FORMAT, channels=CHANNELS,
    #               rate=RATE, input=True,
    #               frames_per_buffer=CHUNK)

   print("recording...")
   frames = record()#[]

   #for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
   #    data = stream.read(CHUNK)
   #    frames.append(data)
   print(frames)
   print("finished recording")


   # stop Recording
   #stream.stop_stream()
   #stream.close()
   audio.terminate()

   # saving wav file 
   waveFile = wave.open(FILENAME, 'wb')
   waveFile.setnchannels(CHANNELS)
   waveFile.setsampwidth(audio.get_sample_size(FORMAT))
   waveFile.setframerate(RATE)
   waveFile.writeframes(b''.join(frames))
   waveFile.close()

   modelpath = "/content/Voice-Authentication-and-Face-Recognition/gmm_models/"

   gmm_files = [os.path.join(modelpath,fname) for fname in 
               os.listdir(modelpath) if fname.endswith('.gmm')]

   models    = [pickle.load(open(fname,'rb')) for fname in gmm_files]

   speakers   = [fname.split("/")[-1].split(".gmm")[0] for fname 
               in gmm_files]
 
   #read test file
   sr,audio = read(FILENAME)
   
   # extract mfcc features
   vector = extract_features(audio,sr)
   log_likelihood = np.zeros(len(models)) 

   #checking with each model one by one
   for i in range(len(models)):
       gmm = models[i]         
       scores = np.array(gmm.score(vector))
       log_likelihood[i] = scores.sum()

   pred = np.argmax(log_likelihood)
   identity = speakers[pred]
  
   # if voice not recognized than terminate the process
   if identity == 'unknown':
           print("Not Recognized! Try again...")
           return
   
   print( "Recognized as - ", identity)


if __name__ == '__main__':
    add_user()
    recognize()

Speak Now...


<IPython.core.display.Javascript object>

Done Recording !
...
Done
Speak your name one more time
Speak Now...


<IPython.core.display.Javascript object>

Done Recording !
...
Done
Speak your name one last time
Speak Now...


<IPython.core.display.Javascript object>

Done Recording !
...
Done


ValueError: ignored

In [ ]:
!cd /content/Voice-Authentication-and-Face-Recognition

In [ ]:
!git clone https://github.com/MohamadMerchant/Voice-Authentication-and-Face-Recognition.git

Cloning into 'Voice-Authentication-and-Face-Recognition'...
remote: Enumerating objects: 322, done.
remote: Total 322 (delta 0), reused 0 (delta 0), pack-reused 322
Receiving objects: 100% (322/322), 44.18 MiB | 26.61 MiB/s, done.
Resolving deltas: 100% (38/38), done.
